# Training of a model 
### **Feedforward Neural Network**

1. **Structure:**
   - **Input Layer:** Accepts the flattened image data (e.g., for 64x64 images, it has 4096 input nodes).
   - **Hidden Layer:** Contains 16 neurons. This layer processes the input data and learns intermediate features.
   - **Output Layer:** Produces the final classification output. For binary classification, this output is a single neuron representing the probability of one class. For multi-class classification, it produces probabilities for each class.

2. **Activation Function:**
   - **Sigmoid Function:** Used for both the hidden and output layers. It transforms the input values to a range between 0 and 1, making it suitable for binary classification and probabilistic outputs.

3. **Training Process:**
   - **Forward Propagation:** Calculates the activations of each layer based on the input data and current weights.
   - **Backward Propagation:** Computes gradients of the loss function with respect to the weights and biases using the chain rule. These gradients are then used to update the weights through gradient descent.
   - **Cost Function:** Uses cross-entropy loss to measure the performance of the network. It calculates how well the predicted probabilities match the true labels.

4. **Learning Mechanism:**
   - **Gradient Descent:** Optimizes the weights and biases by minimizing the cost function. The learning rate determines the step size of each update.


In [1]:
import numpy as np
from PIL import Image
import os
import pandas as pd

# Activation function and its derivative
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def sigmoid_derivative(A):
    return A * (1 - A)

# Initialize parameters
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    return W1, b1, W2, b2

# Forward propagation
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    cache = (A1, A2, Z1, Z2)
    return A2, cache

# Backward propagation
def backward_propagation(X, Y, cache, W2):
    A1, A2, Z1, Z2 = cache
    m = X.shape[1]
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * sigmoid_derivative(A1)
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
    return dW1, db1, dW2, db2

# Update parameters
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

# Compute cost
def compute_cost(A2, Y):
    m = Y.shape[1]
    cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
    return cost

# Train the model
def train(X, Y, n_h, num_iterations=10000, learning_rate=0.5):
    n_x = X.shape[0]
    n_y = Y.shape[0]
    W1, b1, W2, b2 = initialize_parameters(n_x, n_h, n_y)
    
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, W1, b1, W2, b2)
        cost = compute_cost(A2, Y)
        dW1, db1, dW2, db2 = backward_propagation(X, Y, cache, W2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        
        if i % 1000 == 0:
            print(f"Epoch {i // 1000 + 1}: Cost {cost}")
    
    return W1, b1, W2, b2

# Predict function
def predict(X, W1, b1, W2, b2):
    A2, _ = forward_propagation(X, W1, b1, W2, b2)
    return np.argmax(A2, axis=0)  # For multi-class classification

# Load and preprocess images
def prepare_data(image_folder, label_file, image_size=(64, 64)):
    # Load labels
    filenames, labels = load_labels(label_file)
    
    images = []
    for filename in filenames:
        img_path = os.path.join(image_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path)
            img = img.resize(image_size)
            img = np.array(img) / 255.0  # Normalize
            if img.ndim == 3:
                img = img.mean(axis=2)  # Convert to grayscale if needed
            images.append(img.flatten())
        else:
            print(f"Image file not found: {img_path}")
    
    # Convert to numpy arrays
    X = np.array(images).T  # Transpose so each column is an image
    Y = np.array(labels).T  # Transpose to match the expected format
    
    return X, Y

# Load and parse labels
def load_labels(file_path):
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"Label file not found: {file_path}")
    
    df = pd.read_csv(file_path, delimiter=',', header=0, index_col=0)
    filenames = df.index.tolist()
    labels = df.values  # No need to transpose; assumes proper format
    return filenames, labels

# Load and preprocess images
def load_images(folder, image_size=(64, 64)):
    images = []
    labels = []
    
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path)
            img = img.resize(image_size)
            img = np.array(img) / 255.0  # Normalize
            if img.ndim == 3:
                img = img.mean(axis=2)  # Convert to grayscale if needed
            images.append(img.flatten())
            
            # Determine the label from filename
            if "original" in filename.lower():
                labels.append(1)
            else:
                labels.append(0)
        else:
            print(f"Image file not found: {img_path}")
    
    return np.array(images).T, np.array(labels).reshape(1, -1)
 
# Save model parameters
def save_model(W1, b1, W2, b2, filename='model.npz'):
    np.savez(filename, W1=W1, b1=b1, W2=W2, b2=b2)

# Load model parameters
def load_model(filename='model.npz'):
    with np.load(filename) as data:
        W1 = data['W1']
        b1 = data['b1']
        W2 = data['W2']
        b2 = data['b2']
    return W1, b1, W2, b2

# Main execution
if __name__ == "__main__":
    # File paths
    image_folder = r"C:\Users\Hi\Documents\fake_shoe_DL\train"
    label_file = r"C:\Users\Hi\Documents\fake_shoe_DL\_classes_train.csv"
    
    # Prepare data
    X, Y = prepare_data(image_folder, label_file)

    # Neural network architecture
    image_size = (64, 64)
    n_x = image_size[0] * image_size[1]
    n_h = 16  # Number of hidden units
    n_y = Y.shape[0]  # Number of output classes

    num_epoch=30
    # Train the model
    W1, b1, W2, b2 = train(X, Y, n_h, num_iterations=num_epoch*1000, learning_rate=0.5)

    # Save the model parameters
    save_model(W1, b1, W2, b2)

    # Load the model parameters
    W1, b1, W2, b2 = load_model()
    
    # Make predictions on new data
    X_new, _ = load_images(r"C:\Users\Hi\Documents\fake_shoe_DL\test")
    predictions = predict(X_new, W1, b1, W2, b2)
    print(f"Predictions: {predictions}")


Epoch 1: Cost 2.7847073066388672
Epoch 2: Cost 1.6421054794066432
Epoch 3: Cost 1.4675836112184877
Epoch 4: Cost 1.222824147148572
Epoch 5: Cost 1.3073325067953834
Epoch 6: Cost 1.4819194515159795
Epoch 7: Cost 1.0318262499650555
Epoch 8: Cost 0.9114109405732644
Epoch 9: Cost 1.015717305043736
Epoch 10: Cost 1.0015478505095776
Epoch 11: Cost 0.981613518825862
Epoch 12: Cost 0.7739844028128714
Epoch 13: Cost 1.14661907870791
Epoch 14: Cost 0.8895322599089979
Epoch 15: Cost 0.9202550275964778
Epoch 16: Cost 1.274020015650511
Epoch 17: Cost 1.171225096323206
Epoch 18: Cost 1.2084566989072723
Epoch 19: Cost 0.8843307248127367
Epoch 20: Cost 0.8580217012801993
Epoch 21: Cost 2.534163356570839
Epoch 22: Cost 0.8661808908961769
Epoch 23: Cost 1.1504611395644755
Epoch 24: Cost 1.0991320362797825
Epoch 25: Cost 2.3298747048343493
Epoch 26: Cost 1.52129303031343
Epoch 27: Cost 0.5358850579192459
Epoch 28: Cost 2.4051829867554915
Epoch 29: Cost 0.4613127173163226
Epoch 30: Cost 0.615689448825258


# GUI to interact and predict the image 
This script creates a simple **Tkinter-based image classification** application. It loads a pre-trained neural network model from a file and uses it to predict the class of user-uploaded images. The GUI allows users to select an image, which is then displayed and processed. The neural network outputs a class prediction, which is shown to the user. The class labels are predefined and used to interpret the model's output.

In [30]:
import numpy as np
from PIL import Image, ImageTk
import os
import tkinter as tk
from tkinter import filedialog, messagebox

# Activation function and its derivative
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

# Forward propagation
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    return A2

# Load model parameters
def load_model(filename='model.npz'):
    with np.load(filename) as data:
        W1 = data['W1']
        b1 = data['b1']
        W2 = data['W2']
        b2 = data['b2']
    return W1, b1, W2, b2

# Load and preprocess a single image
def preprocess_image(image_path, image_size=(64, 64)):
    img = Image.open(image_path)
    img = img.resize(image_size)
    img = np.array(img) / 255.0  # Normalize
    if img.ndim == 3:
        img = img.mean(axis=2)  # Convert to grayscale if needed
    return img.flatten().reshape(-1, 1)  # Flatten and reshape to match input dimensions

# Make a prediction
def predict(image_path, W1, b1, W2, b2, image_size=(64, 64)):
    X = preprocess_image(image_path, image_size)
    A2 = forward_propagation(X, W1, b1, W2, b2)
    return int(np.argmax(A2))  # Ensure the result is an integer index

# Function to open file dialog and make prediction
def upload_image():
    filepath = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if filepath:
        # Display the image
        img = Image.open(filepath)
        img = img.resize((200, 200))  # Resize for display
        img = ImageTk.PhotoImage(img)
        panel.configure(image=img)
        panel.image = img
        
        # Make a prediction
        try:
            prediction_index = predict(filepath, W1, b1, W2, b2)
            class_label = class_labels[prediction_index]  # Map index to class label
            result_var.set(f"Predicted class: {class_label}")
        except IndexError:
            result_var.set("Prediction index out of range.")
        except Exception as e:
            result_var.set(f"An error occurred: {str(e)}")

# Main application window
root = tk.Tk()
root.title("Image Classification")

# Load model parameters
W1, b1, W2, b2 = load_model('model.npz')

# Define class labels
class_labels = [
    "fake air force",
    "fake jordan 1",
    "original air force",
    "original jordan 1"
]

# Create UI elements
upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack()

panel = tk.Label(root)
panel.pack()

result_var = tk.StringVar()
result_label = tk.Label(root, textvariable=result_var)
result_label.pack()

# Start the Tkinter event loop
root.mainloop()
